In [ ]:
!pip install nltk

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [3]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies"""


In [4]:
# tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# tokenize
tokens = word_tokenize(document.lower())

In [6]:
# build vocab

vocab = {'<unk>': 0, }
tokens_frequency_dict = Counter(tokens)

for token in tokens_frequency_dict.keys():
    if token not in vocab.keys():
      vocab[token]=len(vocab)

len(vocab)

289

In [7]:
# converting tokens(string) value to index(integer) value  in the each sentence
def text_to_indices(sentence, vacab):

    numerical_sentence = []

    for token in sentence:
        if token in vocab:
            numerical_sentence.append(vocab[token])
        else:
          numerical_sentence.append(vocab['<unk>'])

    return numerical_sentence

In [8]:
# extract sentences from data

input_sentences = document.split('\n') # separated each lines of this sentences


In [9]:
input_numerical_sentences=[]

for sentence in input_sentences:
  tokens = word_tokenize(sentence.lower())   # converting each senteces into token in the list

  input_numerical_sentences.append(text_to_indices(tokens, vocab))


In [10]:
# creating training sequence and stroing it

training_sequence = []

for sentence in input_numerical_sentences:

   for i in range(1, len(sentence)):  # range start from 1 bcz we give sequence size minimum 2 words so we can make last word as target or output in training
    training_sequence.append(sentence[:i+1])



In [11]:
print(len(training_sequence))  # training rows


942


In [12]:
# finding longest training sequence
len_sequence = []
for sequence in training_sequence:
  len_sequence.append(len(sequence))

max_len=max(len_sequence)
max_len

62

In [13]:
#  make all sequence size same by 0 padding
padded_training_sequence = []

for sequence in training_sequence:
  needed_padding_len = max_len - len(sequence)
  new_sequence = [0]*needed_padding_len + sequence

  padded_training_sequence.append(new_sequence)


In [14]:
print(len(padded_training_sequence[0]))

62


In [15]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)

In [16]:
padded_training_sequence.shape

torch.Size([942, 62])

In [17]:

X = padded_training_sequence[:, :-1]
y = padded_training_sequence[:, -1]  # assigning last column data as label column

In [18]:
X.shape
y.shape

torch.Size([942])

In [19]:
class CustomDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]

In [20]:
dataset = CustomDataset(X, y)

In [21]:
len(dataset)

dataset[1]

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2]),
 tensor(3))

In [22]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [23]:
class LSTMModel(nn.Module):

  def __init__(self, vocab_size):

    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(input_size=100, hidden_size=150, batch_first=True)
    self.fc = nn.Linear(in_features=150, out_features=vocab_size)

  def forward(self, X):
    embedded = self.embedding(X)
    intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))

    return output


In [24]:
model = LSTMModel(len(vocab))

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [26]:
model.to(device)

LSTMModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=289, bias=True)
)

In [27]:
epochs = 50
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [28]:
# training loop

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in dataloader:

    batch_x , batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 165.4335
Epoch: 2, Loss: 146.9590
Epoch: 3, Loss: 135.2757
Epoch: 4, Loss: 123.1196
Epoch: 5, Loss: 111.3805
Epoch: 6, Loss: 99.8131
Epoch: 7, Loss: 89.2862
Epoch: 8, Loss: 78.9974
Epoch: 9, Loss: 70.0484
Epoch: 10, Loss: 61.9188
Epoch: 11, Loss: 54.8896
Epoch: 12, Loss: 47.8185
Epoch: 13, Loss: 42.0582
Epoch: 14, Loss: 36.8710
Epoch: 15, Loss: 32.6417
Epoch: 16, Loss: 28.6807
Epoch: 17, Loss: 25.3046
Epoch: 18, Loss: 22.3674
Epoch: 19, Loss: 20.1041
Epoch: 20, Loss: 17.8354
Epoch: 21, Loss: 16.0774
Epoch: 22, Loss: 14.7359
Epoch: 23, Loss: 13.1956
Epoch: 24, Loss: 12.1966
Epoch: 25, Loss: 11.2843
Epoch: 26, Loss: 10.3420
Epoch: 27, Loss: 9.8029
Epoch: 28, Loss: 8.9778
Epoch: 29, Loss: 8.4168
Epoch: 30, Loss: 7.9950
Epoch: 31, Loss: 7.7856
Epoch: 32, Loss: 7.1614
Epoch: 33, Loss: 6.9576
Epoch: 34, Loss: 6.6583
Epoch: 35, Loss: 6.5329
Epoch: 36, Loss: 6.2092
Epoch: 37, Loss: 5.8679
Epoch: 38, Loss: 5.6958
Epoch: 39, Loss: 5.6529
Epoch: 40, Loss: 5.5640
Epoch: 41, Loss: 5

In [69]:
# prediction

def prediction(model, vocab, text):

  # tokenize input text
  tokenized_text = word_tokenize(text.lower())


  # text to numerical indices
  numerical_text = text_to_indices(tokenized_text, vocab)


  # padding
  padding_len = 61 - len(numerical_text)
  padded_numerical_text = torch.tensor([0]*padding_len + numerical_text, dtype=torch.long).unsqueeze(0)


  # send to model
  output = model(padded_numerical_text.to(device))


  # predicted index
  value, index = torch.max(output, dim=1)

  # merge with text
  return text + ' ' +  list(vocab.keys())[index]


In [71]:
prediction(model, vocab, "data")

'data analysis'

In [77]:
# to print more next word
num_tokens = 10
input_text = "This is to clarify that"

for i in range(num_tokens):
  next_word = prediction(model, vocab, input_text)
  print(next_word)
  input_text = next_word

This is to clarify that placement
This is to clarify that placement assistance
This is to clarify that placement assistance does
This is to clarify that placement assistance does not
This is to clarify that placement assistance does not mean
This is to clarify that placement assistance does not mean placement
This is to clarify that placement assistance does not mean placement guarantee
This is to clarify that placement assistance does not mean placement guarantee .
This is to clarify that placement assistance does not mean placement guarantee . so
This is to clarify that placement assistance does not mean placement guarantee . so we
